In [14]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2

import sys

sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
from clean_helpers import *

take_full = False
test_locally = True

# Specify here what cleaning functions you want to use
cleaning_options = ['clean_new_line', 'clean_tags', 'lowercase', \
                    'clean_punctuation', 'remove_stopwords']


clean = {
    "clean_new_line": clean_new_line,
    "lowercase": lowercase,
    "lemmatize": "",
    "remove_stopwords": remove_stopwords,
    "translate": "",
    "clean_punctuation": clean_punctuation,
    "clean_tags" : clean_tags
}


# algorithm_used = ""
# algorithm = {
#     "naive_bayes": ,
#     "logistic_regression": ,
#     "svm": ,
#     "lstm":,
#     "fasttext":,
#     "cnn": ,
# }

# options = []
# additional_options = {
#     "count_frequency": ,
#     "count_ngrams": ,
    
# }


In [61]:
input_file_pos = 'Data/train_pos.txt'
if take_full:
    input_file_pos = 'Data/train_pos_full.txt'
  
input_file_neg = 'Data/train_neg.txt'
if take_full:
    input_file_neg = 'Data/train_neg_full.txt'
    
list_of_pos_sentences = []
with open(input_file_pos, 'r') as f:
    for line in f:
        list_of_pos_sentences.append(line)
 
list_of_neg_sentences = []
with open(input_file_neg, 'r') as f:
    for line in f:
        list_of_neg_sentences.append(line)

In [82]:
from data_handling import build_sentences

df = build_sentences(list_of_pos_sentences, list_of_neg_sentences)

df = df[0:1000]

for clean_option in cleaning_options:
    df = clean[clean_option](df)

df.head()

['dunno', 'justin', 'read', 'mention', 'justin', 'god', 'knows', 'hope', 'follow', '#believe', '15']
['logic', 'dumb', 'even', 'crop', 'name', 'photo', 'tsk']
['put', 'casper', 'box', '!', 'looved', 'battle', '!', '#crakkbitch']
['thanks', 'sir', 'trip', 'lil', 'mama', '...', 'keep', 'doin', 'ya', 'thang', '!']
['visiting', 'brother', 'tmr', 'bestest', 'birthday', 'gift', 'eveerrr', '!', '!', '!']
['yay', '!', '!', '#lifecompleted', 'tweet', '/', 'facebook', 'let', 'know', 'please']
['#1dnextalbumtitle', 'feel', '/', 'rollercoaster', 'life', 'song', 'cocept', 'life', '#yolo', 'becoming', 'famous', '?', '<3', '14', '#followmeplz', '!', '<3', 'x15']
['workin', 'hard', 'hardly', 'workin', 'rt', "hardee's", 'future', 'coworker']
['saw', "i'll", 'replying', 'bit']
['belong']
['anddd', 'cheer', '#nationals2013', '?']
['send', 'invitation', 'shop', 'on-line', '!', 'find', 'everything', 'need', '-', 'without', 'leaving', 'home', '...']
['woke', 'finna', 'go', 'church']
['agreed', '!', '12', 'd

,sentence,label
0,dunno justin read mention justin god knows hop...,1
1,logic dumb even crop name photo tsk,1
2,put casper box ! looved battle ! #crakkbitch,1
3,thanks sir trip lil mama ... keep doin ya thang !,1
4,visiting brother tmr bestest birthday gift eve...,1


In [57]:
df.head()

,sentence,label
0,i dunno justin read my mention or not . only j...,1
1,"because your logic is so dumb , i won't even c...",1
2,""" just put casper in a box ! "" looved the batt...",1
3,thanks sir > > don't trip lil mama ... just ke...,1
4,visiting my brother tmr is the bestest birthda...,1


In [ ]:
model = algorithm[algorithm_used]

In [ ]:
y_pred = predict(model, test_x)

In [ ]:
output_file = "output/output.csv"
if not test_locally:
    write(y_pred, output_file)